In [1]:
# load model
import os
os.environ["HF_HOME"] = "/data/cache/huggingface"
# check env
print('HF_HOME: {}'.format(os.environ["HF_HOME"]))

import guidance
import torch

MODEL = 'huggyllama/llama-65b'
# use LLaMa 65b
print('Loading LLaMa 65b')
guidance.llm = guidance.llms.transformers.LLaMA(MODEL, device_map="auto", token_healing=True, torch_dtype=torch.bfloat16)

HF_HOME: /data/cache/huggingface
Loading LLaMa 65b


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [14]:
EXAMPLES = [{
    "problem": "Find the domain of the expression $\\frac{\sqrt{x-2}}{\sqrt{5-x}}$.}",
    "solution": """The expressions inside each square root must be non-negative. Therefore,
$x-2 \ge 0$, so $x\ge2$, and $5 - x \ge 0$, so $x \le 5$. Also, the denominator
cannot be equal to zero, so $5-x>0$, which gives $x<5$. Therefore, the domain of
the expression is $\\boxed{[2,5)}$.""",
    "description": "",
    "answer": "[2,5)",
}]

INSTRUCTION_W_DESC = """
### Instruction:
You are a top-notch mathematician. Your task is to provide the solution of the given problem, with the final answer boxed by \\boxed{}.
Some tasks have a description of the picture to help you understand the problem if "Description" is not empty.
----

"""

INSTRUCTION_WO_DESC = """
### Instruction:
You are a top-notch mathematician. Your task is to provide the solution of the given problem, with the final answer boxed by \\boxed{}.
----

"""

PATTERN_W_DESC = INSTRUCTION_W_DESC + """
{{~! display the few-shot examples ~}}
{{~#each examples}}
### Input:
Problem: {{this.problem}}
Description: {{this.description}}

### Response:
Solution: {{this.solution}}
Answer: {{this.answer}}
---
{{~/each}}

{{~! place the real question at the end }}
### Input:
Problem: {{problem}}
Description: {{description}}

### Response:
Solution: {{gen "solution" temperature=0. max_tokens=400 stop='Answer'}}
Answer: {{gen "answer" temperature=0. max_tokens=50 stop='\n'}}
"""

PATTERN_WO_DESC = INSTRUCTION_WO_DESC + """
{{~! display the few-shot examples ~}}
{{~#each examples}}
### Input:
Problem: {{this.problem}}

### Response:
Solution: {{this.solution}}
Answer: {{this.answer}}
---
{{~/each}}

{{~! place the real question at the end }}
### Input:
Problem: {{problem}}

### Response:
Solution: {{gen "solution" temperature=0. max_tokens=400 stop='Answer'}}
Answer: {{gen "answer" temperature=0. max_tokens=50 stop='\n'}}
"""

In [15]:
# we can pre-define valid option sets
valid_judgement = ["True", "False", "Unknown"]

# program
program = guidance(PATTERN_WO_DESC)


# try once
try: 
    out = program(
        examples=EXAMPLES,
        problem="Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).",
        description="",
    )
except Exception as e:
    print(e)

print(out['answer'])

Stop program 
### Instruction:
You are a top-notch mathematician. Your task is to provide the solution of the given problem, with the final answer boxed by \boxed{}.
---- 
### Input:
Problem: Find the domain of the expression $\frac{\sqrt{x-2}}{\sqrt{5-x}}$.} 

### Response:
Solution: The expressions inside each square root must be non-negative. Therefore,
$x-2 \ge 0$, so $x\ge2$, and $5 - x \ge 0$, so $x \le 5$. Also, the denominator
cannot be equal to zero, so $5-x>0$, which gives $x<5$. Therefore, the domain of
the expression is $\boxed{[2,5)}$. 
Answer: [2,5) 
--- 
### Input:
Problem: Let \[f(x) = \left\{
\begin{array}{cl} ax+3, &\text{ if }x>2, \\
x-5 &\text{ if } -2 \le x \le 2, \\
2x-b &\text{ if } x <-2.
\end{array}
\right.\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper). 

### Response:
Solution: The function is continuous if the left and right limits at $x=2$ and $x=-2$ are equal to the function values at these points.

At $x=2$, the left limit is $2a+3$ and the right limit is $a+3$. Therefore, $2a+3=a+3$, so $a=0$.

At $x=-2$, the left limit is $-2b+3$ and the right limit is $2b-2$. Therefore, $-2b+3=2b-2$, so $b=1$.

Therefore, $a+b=0+1=1$.
 
Answer: 1

1
